In [13]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
from tqdm import tqdm, trange
from time import sleep
from collections import Counter
import numpy as np
import re

In [2]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

Did not use initialization regex that was passed: _context_layer._module.weight_ih.*
Did not use initialization regex that was passed: _context_layer._module.weight_hh.*


In [14]:
predictor.cuda_device

-1

In [7]:
predictor.predict(document="The woman reading a newspaper sat on the bench with her dog.")

{'top_spans': [[0, 4], [5, 5], [7, 8], [10, 10], [10, 11]],
 'antecedent_indices': [[0, 1, 2, 3, 4],
  [0, 1, 2, 3, 4],
  [0, 1, 2, 3, 4],
  [0, 1, 2, 3, 4],
  [0, 1, 2, 3, 4]],
 'predicted_antecedents': [-1, -1, -1, 0, -1],
 'document': ['The',
  'woman',
  'reading',
  'a',
  'newspaper',
  'sat',
  'on',
  'the',
  'bench',
  'with',
  'her',
  'dog',
  '.'],
 'clusters': [[[0, 4], [10, 10]]]}

In [8]:
shawshank_parsed = open("annotated-data/shawshank.script_parsed.txt").read().strip().split("\n")
tags, texts = [], []

for line in shawshank_parsed:
    tag, text = line[0], line[3:].strip()
    tags.append(tag)
    texts.append(text)

In [9]:
tags[:10]

['M', 'M', 'M', 'M', 'M', 'M', 'S', 'N', 'S', 'N']

In [10]:
texts[:5]

['THE SHAWSHANK REDEMPTION',
 'by',
 'Frank Darabont',
 'Based upon the story',
 'Rita Hayworth and Shawshank Redemption']

In [12]:
Counter(tags)

Counter({'M': 6, 'S': 38, 'N': 96, 'C': 169, 'D': 169, 'E': 47})

In [21]:
predicted_clusters = []

for text in tqdm(texts):
    try:
        predicted_cluster = predictor.predict(document=text)
    except Exception:
        predicted_cluster = None
    predicted_clusters.append(predicted_cluster)


100%|██████████| 525/525 [02:12<00:00,  3.95it/s]


In [22]:
n_none_clusters = 0

for c in predicted_clusters:
    if c is None:
        n_none_clusters += 1

print(n_none_clusters)

202


In [23]:
len(predicted_clusters)

525

In [25]:
for tag, text, predicted_cluster in zip(tags, texts, predicted_clusters):
    if predicted_cluster is None:
        print(tag, text)

M by
M Frank Darabont
C WOMAN
E O.S.
C WOMAN
E O.S.
C D.A.
E O.S.
C ANDY
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
E Andy nods
C ANDY
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
C ANDY
E softly
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
C D.A.
C ANDY
C D.A.
C ANDY
C D.A.
C D.A.
C D.A.
C VOICE
E O.S.
C JUDGE
D Sit.
C RED
C RED
E V.O.
C RED
E V.O.
C RED
E V.O.
C CON
C RED
E V.O.
C RED
E V.O.
C TOWER GUARD
C HADLEY
C RED
C HEYWOOD
C JIGGER
C FLOYD
C RED
C FLOYD
C RED
C FLOYD
C HEYWOOD
C ERNIE
C HEYWOOD
C FLOYD
C HEYWOOD
C RED
C RED
E V.O.
C SKEET
C RED
C SNOOZE
C RED
C VOICE
E amplified
C HADLEY
C NORTON
C CON
C HADLEY
C NORTON
C HADLEY
C HADLEY
C HADLEY
C DOCTOR
C CHAPLAIN
E O.S.
C RED
E V.O.
C RED
E V.O.
C RED
E V.O.
C RED
E V.O.
C GUARD
C RED
E V.O.
C VARIOUS VOICES
C RED
E V.O.
C HEYWOOD
E O.S.
C HEYWOOD
E O.S.
C HEYWOOD
C VOICES
C VOICE
E O.S.
C HADLEY
C VOICE
E O.S.
C HADLEY
C HADLEY
C HADLEY
C HADLEY
E peers around
C RED
E V.O.
C BROOKS
C ANDY
C BROOKS
C BROOKS
C 

In [27]:
for tag, text, predicted_cluster in zip(tags, texts, predicted_clusters):
    if predicted_cluster and predicted_cluster["clusters"]:
        document = predicted_cluster["document"]
        clusters = predicted_cluster["clusters"]
        
        print("TEXT ==>")
        print(text)
        print()
        
        print("CLUSTERS ==>")
        print()
        
        for i, cluster in enumerate(clusters):
            print(f"\tCLUSTER{i + 1}")
            for j, (beg, end) in enumerate(cluster):
                span = " ".join(document[k] for k in range(beg, end + 1))
                print(f"\t\t{j + 1:3d}. {span}")
            print()
        print("\n***************************************************************************************************************************************************\n")

TEXT ==>
A dark, empty room. The door bursts open. A MAN and WOMAN enter, drunk and giggling, horny as hell. No sooner is the door shut than they're all over each other, ripping at clothes, pawing at flesh, mouths locked together. He gropes for a lamp, tries to turn it on, knocks it over instead. Hell with it. He's got more urgent things to do, like getting her blouse open and his hands on her breasts. She arches, moaning, fumbling with his fly. He slams her against the wall, ripping her skirt. We hear fabric tear. He enters her right then and there, roughly, up against the wall. She cries out, hitting her head against the wall but not caring, grinding against him, clawing his back, shivering with the sensations running through her. He carries her across the room with her legs wrapped around him. They fall onto the bed. CAMERA PULLS BACK, exiting through the window, traveling smoothly outside...

CLUSTERS ==>

	CLUSTER1
		  1. The door
		  2. the door

	CLUSTER2
		  1. A MAN and WOMAN
